In [1]:
!pip install transformers
! pip install kagglehub
! pip install bitsandbytes transformers accelerate
!pip install accelerate>=0.26.0
!pip install implicit
!pip install datasets
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 203.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 420.3 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 243.9 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Looking in indexes: https://pypi.org/simple, https

In [ ]:
import os
os.kill(os.getpid(), 9)  # This will force a Colab restart


1. Download and read dataset.
   
It contains 3 tables - 'Users.csv', 'Ratings.csv', 'Books.csv'.

In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("arashnic/book-recommendation-dataset")

print("Path to dataset files:", path)

Path to dataset files: /home/jovyan/.cache/kagglehub/datasets/arashnic/book-recommendation-dataset/versions/3


In [4]:
 path = '/home/jovyan/.cache/kagglehub/datasets/arashnic/book-recommendation-dataset/versions/3'

In [5]:
import os

files = os.listdir(path)
print(files)

['DeepRec.png', 'Users.csv', 'Ratings.csv', 'Books.csv', 'classicRec.png', 'recsys_taxonomy2.png']


In [6]:

import pandas as pd

books = pd.read_csv(os.path.join(path, 'Books.csv'))
books.head(5)


/tmp/ipykernel_85/66192168.py:3: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv(os.path.join(path, 'Books.csv'))


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [29]:
ratings[ratings['ISBN']=='0195153448']

,User-ID,ISBN,Book-Rating,user_index,book_index
9561,2,0195153448,0,850,8967


In [35]:
ratings[ratings['User-ID']==276729]

,User-ID,ISBN,Book-Rating,user_index,book_index
3,276729,052165615X,3,3,3
4,276729,0521795028,6,3,4


The code below generates book descriptions using a large language model (LLM), specifically a model from the Meta-LLaMA family (Llama-3.2-1B). It utilizes the Hugging Face transformers library for model loading and inference and processes book metadata (title, author) to create short descriptions.

It loads a pre-trained tokenizer for the selected model.
Sets the padding token to be the same as the model's end-of-sequence (EOS) token, ensuring that the model does not produce unnecessary padding errors.

The text-generation pipeline is working in batches of size 128

Function take as input 
-- Book title and author.
Creates a prompt: "Book description: '{book_title}' by {book_author}. Summary:"

Calls the LLM (via generator) to generate text with:
-- max_length=150: The maximum length of generated text.
-- truncation=True: Ensures the output does not exceed the limit.
-- num_return_sequences=1: Generates only one description per request.

In [9]:
from transformers import pipeline
import multiprocessing
from transformers import AutoTokenizer, pipeline
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from datasets import Dataset


#model_name = "meta-llama/Llama-2-7b-chat-hf"

#model_name = "gpt2"


# Define the model name
model_name = "meta-llama/Llama-3.2-1B"

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side='left')

# ✅ Set `pad_token_id` after loading the tokenizer
tokenizer.pad_token = tokenizer.eos_token


# Load the text-generation pipeline
generator = pipeline("text-generation", model=model_name, tokenizer=tokenizer, batch_size=128)

print("✅ Tokenizer initialized successfully!")

model = AutoModelForCausalLM.from_pretrained(model_name).to("cuda") #move to cuda.


def generate_book_description(book_title, book_author):
    """Generates a book description using a pre-trained LLM."""

        #generator = pipeline("text2text-generation", model=model_name) # corrected pipeline.
    #prompt = f"Write a brief book description for a novel titled: {book_title}"
    #prompt = f"Book description: '{row['Book-Title']}' by {row['Book-Author']}. Summary:"
    prompt = f"Book description: '{book_title}' by {book_author}. Summary:"

    try:
        generated_text = generator(prompt, max_length=150, truncation=True, num_return_sequences=1)[0]['generated_text']
        description = generated_text.split("Summary:")[1].strip()
        if len(description) < 3:
            return "Failed to generate a usable description"
        return description

        #description = generated_text.split(book_title + ": ")[1] #clean up the output.
        #return description
    except IndexError:
        return "Could not generate a description."


def generate_descriptions(batch):
    """Generates book descriptions for a batch of inputs."""
    prompts = [f"Book description: '{title}' by {author}. Summary:" for title, author in zip(batch['Book-Title'], batch['Book-Author'])]
    inputs = tokenizer(prompts, return_tensors="pt", padding=True, truncation=True).to("cuda")
    outputs = model.generate(**inputs, max_length=150, num_return_sequences=1)
    descriptions = [tokenizer.decode(output, skip_special_tokens=True).split("Summary:")[1].strip() for output in outputs]
    batch['Description'] = descriptions
    return batch

#for title in book_titles:
'''descriptions = []

for index, row in books.iterrows():
    description = generate_book_description(row)
    descriptions.append(description)
    #print(f"Title: {row['Book-Title']}")
    #print(f"Description: {description}\n")

books['Description'] = '''


def process_row(row):
    return generate_book_description(row['Book-Title'], row['Book-Author'])

def parallel_process(books):
    
    descriptions = process_row([row for _, row in books.iterrows()])
    books['Description'] = descriptions
    return books



#books = parallel_process(books)


books = books.dropna(subset=['Book-Title', 'Book-Author'])

books['Book-Title'] = books['Book-Title'].astype(str)
books['Book-Author'] = books['Book-Author'].astype(str)
# Assuming 'books' is your pandas DataFrame
books['Year-Of-Publication'] = pd.to_numeric(books['Year-Of-Publication'], errors='coerce')
books = books.dropna(subset=['Year-Of-Publication']) # Drop rows that failed conversion.



# Generate descriptions and add as a new column
'''books['Description'] = books.apply(
    lambda row: generate_book_description(row['Book-Title'], row['Book-Author']),
    axis=1
)'''

dataset = Dataset.from_pandas(books)
tokenized_dataset = dataset.map(lambda examples: tokenizer(examples['Book-Title'], examples['Book-Author'], truncation=True, padding=True), batched=True)


processed_dataset = tokenized_dataset.map(generate_descriptions, batched=True, batch_size=128) #adjust batch size.
processed_df = processed_dataset.to_pandas()

print(processed_df)

Device set to use cuda:0


✅ Tokenizer initialized successfully!


/tmp/ipykernel_356/770252573.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  books['Book-Title'] = books['Book-Title'].astype(str)
/tmp/ipykernel_356/770252573.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  books['Book-Author'] = books['Book-Author'].astype(str)
/tmp/ipykernel_356/770252573.py:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.

              ISBN                                         Book-Title  \
0       0195153448                                Classical Mythology   
1       0002005018                                       Clara Callan   
2       0060973129                               Decision in Normandy   
3       0374157065  Flu: The Story of the Great Influenza Pandemic...   
4       0393045218                             The Mummies of Urumchi   
...            ...                                                ...   
271350  0440400988                         There's a Bat in Bunk Five   
271351  0525447644                            From One to One Hundred   
271352  006008667X  Lily Dale : The True Story of the Town that Ta...   
271353  0192126040                        Republic (World's Classics)   
271354  0767409752  A Guided Tour of Rene Descartes' Meditations o...   

                 Book-Author  Year-Of-Publication  \
0         Mark P. O. Morford               2002.0   
1       Richard B

The dataframe with books descriptions is saved via pickle.


In [12]:
processed_df.to_pickle('books_w_descript.pkl')



Below are represented some examples of denerated book descriptions.


In [14]:
import pandas as pd

processed_df = pd.read_pickle('books_w_descript.pkl')
import pandas as pd

# Set the option to display the full content of each column
pd.set_option('display.max_colwidth', None)


print(processed_df['Description'])


0                                                                                                                                                         'Classical Mythology' by Mark P. O. Morford. This book is about the myths of the Greeks and Romans. It covers the myths of all the gods and goddesses and includes the myths of the Titans, the Olympians and the Muses. The book also includes a chapter on the Titans and the Olympians, as well as the myth of the Muses. It also includes a chapter on the Titans and the Olympians.
1                                                                                      Clara Callan was born in 1888, in a small town in the Midwest. Her father was a successful farmer, and her mother was a beautiful, intelligent woman. When she was only three years old, her father died, and her mother married a man who was abusive and unfaithful. Clara's mother died when she was only five, and her father remarried. Clara was sent to live with her stepmother and st